In [1]:
# API : Application Programming Interface => set of rules and tools that allow different software applications to communicate with each other.
# Enable developers to fetch data from a server, send data to a server, or perform other operations.
# Make a request and get the response from the API
# API sends back a response, which is usually in JSON format.

https://api.openweathermap.org/data/2.5/weather?q=London&appid=YOUR_API_KEY

SyntaxError: invalid syntax (2941487811.py, line 6)

In [2]:
API_KEY = "a7b9a036133578abcbe245cadff3e82c"

# Fetching data from the OpenWeatherMap API
import requests

city = "London"
url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}"

response = requests.get(url)

if response.status_code == 200:
    weather_data = response.json()
    print(weather_data)
else:
    print(f"Error: Unable to fetch data for {city}. Status code: {response.status_code}")

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 292.95, 'feels_like': 293, 'temp_min': 292.01, 'temp_max': 294.27, 'pressure': 1024, 'humidity': 77, 'sea_level': 1024, 'grnd_level': 1020}, 'visibility': 10000, 'wind': {'speed': 4.12, 'deg': 80}, 'clouds': {'all': 100}, 'dt': 1750401903, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1750390975, 'sunset': 1750450873}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


In [ ]:
# Weather App using OpenWeatherMap API

import requests

weather_record_file = "./assets/weather_records.json"

# Step 1: API Setup
API_KEY = "a7b9a036133578abcbe245cadff3e82c"
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
BASE_FORECAST_URL = "https://api.openweathermap.org/data/2.5/forecast"
# Step 2: Function to get weather data
def get_weather(city):
    try:
        url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            weather = {
                "City": data["name"],
                "Temperature": f'{data["main"]["temp"]}C',
                "Weather": data["weather"][0]["description"].title(),
                "Humidity": f'{data["main"]["humidity"]}%',
                "Wind Speed": f'{data["wind"]["speed"]} m/s',
            }
            return weather
        elif response.status_code == 404:
            print(f"Error: City '{city}' not found.")
        else:
            print(f"Error: Unable to fetch data for {city}. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return None

# Step 3: Display Weather Information
def display_weather(weather):
    print("\n------ Weather Information: ------")
    for key, value in weather.items():
        print(f"{key}: {value}")
        
# Step bonus: Save weather data to a file
import json
from datetime import datetime

def load_weather_records():
    try:
        with open(weather_record_file, 'r') as file:
            records = json.load(file)
        return records
    except FileNotFoundError:
        print("Weather records file not found. Starting with an empty record.")
    except json.JSONDecodeError:
        print("Error reading weather records. The file may be corrupted.")
    return []
    
def save_weather_to_file(weather):
    records = load_weather_records()
    records.append(weather)
    try:
        with open(weather_record_file, 'w') as file:
            json.dump(records, file)
        print("Weather data saved successfully.")
    except Exception as e:
        print(f"An error occurred while saving data: {e}")

# Step bonus: Load and display saved weather records
def display_saved_records():
    records = load_weather_records()
    if records:
        print("\n------ Saved Weather Records: ------")
        for record in records:
            print("\n-------------------------------")
            print(f"City: {record['City']}")
            print(f"Temperature: {record['Temperature']}")
            print(f"Weather: {record['Weather']}")
            print(f"Humidity: {record['Humidity']}")
            print(f"Wind Speed: {record['Wind Speed']}")
    else:
        print("No saved weather records found.")

# Step bonus : Display next 5 day forecast
def get_forecast(city):
    try:
        url_current = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
        response_current = requests.get(url_current)
        current_date = None
        if response_current.status_code == 200:
            data_current = response_current.json()
            # Get current weather date (UTC)
            current_dt = data_current.get("dt")
            if current_dt:
                current_date = datetime.utcfromtimestamp(current_dt).strftime("%Y-%m-%d")
                print(f"Current weather date: {current_date}")
        url_forecast = f"{BASE_FORECAST_URL}?q={city}&appid={API_KEY}&units=metric"
        response_forecast = requests.get(url_forecast)
        if response_forecast.status_code == 200:
            data = response_forecast.json()
            forecast = []
            for item in data["list"]:
                if current_date != item["dt_txt"].split(" ")[0] and len(forecast) < 5:
                    current_date = item["dt_txt"].split(" ")[0]
                    forecast.append({
                        "Date": item["dt_txt"].split(" ")[0],
                        "Temperature": f'{item["main"]["temp"]}C',
                        "Weather": item["weather"][0]["description"].title(),
                        "Humidity": f'{item["main"]["humidity"]}%',
                        "Wind Speed": f'{item["wind"]["speed"]} m/s',
                    })
                elif len(forecast) >= 5:
                    break
                else:
                    continue
            return forecast
        elif response_forecast.status_code == 404:
            print(f"Error: City '{city}' not found.")
        else:
            print(f"Error: Unable to fetch data for {city}. Status code: {response_forecast.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return None

# Step bonus : Show menu
def show_menu():
    print("\n------- Weather App Menu -------")
    print("1. Get Weather Information")
    print("2. Display Saved Weather Records")
    print("3. Exit")

def show_menu_city(city=None):
    print(f"\n------- {city if city else "City"} Menu -------")
    print("1. Get Current Weather")
    print("2. Get 5-Day Forecast")
    print("3. Back to Main Menu")
    
    
# Step 4: Main Program Loop
while True:
    show_menu()
    choice = input("Choose an option (1-3): ").strip()
    if choice == '1':
        while True:
            city = input("Enter city name: ").strip()
            show_menu_city()
            city_choice = input(f"Choose an option (1-3): ").strip()
            if city_choice == '1':
                weather = get_weather(city)
                if weather:
                    display_weather(weather)
                    input("Do you want to save this weather data? (yes/no): ").strip().lower()
                    if input() == 'yes':
                        save_weather_to_file(weather)
                    else:
                        print("Weather data not saved.")
            elif city_choice == '2':
                city = input("Enter city name for forecast: ").strip()
                forecast = get_forecast(city)
                if forecast:
                    print("\n------ 5-Day Forecast: ------")
                    for day in forecast:
                        print("\n-------------------------------")
                        print(f"Date: {day['Date']}")
                        print(f"Temperature: {day['Temperature']}")
                        print(f"Weather: {day['Weather']}")
                        print(f"Humidity: {day['Humidity']}")
                        print(f"Wind Speed: {day['Wind Speed']}")
            elif city_choice == '3':
                break
            else:
                print("Invalid choice. Please try again.")
    if choice == '2':
        display_saved_records()
    elif choice == '3':
        print("Exiting the Weather App. Goodbye!")
        break
    else:
        print("Invalid choice. Please try again.")


------- Weather App Menu -------
1. Get Weather Information
2. Display Saved Weather Records
3. Exit



------- City Menu -------
1. Get Current Weather
2. Get 5-Day Forecast
3. Back to Main Menu
Invalid choice. Please try again.

------- Weather App Menu -------
1. Get Weather Information
2. Display Saved Weather Records
3. Exit
Exiting the Weather App. Goodbye!
